In [1]:
import time
from itertools import combinations
import numpy as np
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
import networkx as nx
import sys, os

In [2]:
def matrix_pattern(pattern):
    n = len(pattern)
    pattern = np.reshape(pattern, (1, len(pattern)))
    X = (np.transpose(pattern) @ pattern) / 2
    for i in range(n):
         X[i, i] = pattern[0, i]
    return X

def matmul(A, B):
    assert len(A) == len(B)
    n = len(A)
    return sum([sum([A[i][j]*B[i][j] for j in range(n)]) for i in range(n)])

def matminus(A, B):
    assert len(A) == len(B)
    n = len(A)
    return [[A[i][j] - B[i][j] for j in range(n)] for i in range(n)]

In [3]:
def ortools_map_lp(J, h, infected_nodes):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #solver.SetNumThreads(1082000)
    
    # prepare data
    nodes = list(range(len(J)))
    G = nx.from_numpy_matrix(J)
    edges = list(G.edges())
    not_infected_nodes = list(set(nodes).difference(set(infected_nodes)))
    H = G.subgraph(not_infected_nodes)
    not_infected_edges = list(H.edges())
    
    
    node_beliefs = []
    for node in nodes:
        zero_belief = solver.NumVar(0, 1, 'zero_belief_of_node_{}'.format(node))
        one_belief = solver.NumVar(0, 1, 'one_belief_of_node_{}'.format(node))
        node_beliefs.append([zero_belief, one_belief])

    edge_beliefs = []
    for edge in edges:
        zz_belief = solver.NumVar(0, 1, '00_belief_of_edge_{}'.format(edge))
        oz_belief = solver.NumVar(0, 1, '01_belief_of_edge_{}'.format(edge))
        zo_belief = solver.NumVar(0, 1, '10_belief_of_edge_{}'.format(edge))
        oo_belief = solver.NumVar(0, 1, '11_belief_of_edge_{}'.format(edge))
        edge_beliefs.append([[zz_belief, oz_belief], [zo_belief, oo_belief]])

    objective = 0.0

    # add infected nodes
    for node in infected_nodes:
        solver.Add(node_beliefs[node][0] == 0.0)
        solver.Add(node_beliefs[node][1] == 1.0)

    # add pairwise interactions
    for i in range(len(edges)):
        #i = edges.index(edge)
        edge = edges[i]
        a = edge[0]
        b = edge[1]
        objective += J[a, b] * edge_beliefs[i][1][1] / 2
        objective += J[a, b] * edge_beliefs[i][0][0] / 2
        objective -= J[a, b] * edge_beliefs[i][1][0] / 2 
        objective -= J[a, b] * edge_beliefs[i][0][1] / 2
        
        #objective += J[a, b] * (2 * x[a] - 1) * (2 * x[b] - 1) / 2
        
    # Add field
    for node in nodes:
        objective += h[node] * (node_beliefs[node][1] - node_beliefs[node][0])

    # normalization constraints
    for node in not_infected_nodes:
        solver.Add(node_beliefs[node][0] + node_beliefs[node][1] == 1)

    # marginalization constraints
    for edge in edges:
        a = edge[0]
        b = edge[1]
        marginal_left_0 = edge_beliefs[edges.index(edge)][0][0] + \
                          edge_beliefs[edges.index(edge)][0][1]
        solver.Add(marginal_left_0 == node_beliefs[a][0])

        marginal_left_1 = edge_beliefs[edges.index(edge)][1][0] + \
                          edge_beliefs[edges.index(edge)][1][1]
        solver.Add(marginal_left_1 == node_beliefs[a][1])

        marginal_right_0 = edge_beliefs[edges.index(edge)][0][0] + \
                           edge_beliefs[edges.index(edge)][1][0]
        solver.Add(marginal_right_0 == node_beliefs[b][0])

        marginal_right_1 = edge_beliefs[edges.index(edge)][0][1] + \
                           edge_beliefs[edges.index(edge)][1][1]
        solver.Add(marginal_right_1 == node_beliefs[b][1])

    solver.Maximize(objective)
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        node_beliefs_result = [[node_beliefs[node][0].solution_value(), node_beliefs[node][1].solution_value()] for node in range(len(nodes))]
        return node_beliefs_result
    else:
        print('The problem does not have an optimal solution.')
        return None


In [4]:
def l1_project(J, h, secure=1, margin=0.0):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    n = len(J)
    K = [[solver.NumVar(-1, solver.infinity(), 'K_{}{}'.format(i, j)) for i in range(n)] for j in range(n)]
    
    constraints = []
    for i in range(n):
        solver.Add(K[i][i] == h[i])
        for j in range(n):
            if i != j:
                solver.Add(K[i][j] <= J[i, j])
                solver.Add(K[i][j] >= 0)

    # add all patterns with single infection
    cascade_pattern = np.ones(n)
    X1 = matrix_pattern(cascade_pattern)

    for k in range(1, secure+1):
        combs = combinations(range(n), k)
        for comb in combs:
            pattern = -1*np.ones(n)
            for el in comb:
                pattern[el] = 1
            X = matrix_pattern(pattern)
            solver.Add(matmul(K, X) >= matmul(K, X1) + margin)

    # eliminate cascade case
    healthy_pattern = -1*np.ones(n)
    X0 = matrix_pattern(cascade_pattern)
    constraints += [matmul(K, X0) >= matmul(K, X1) + margin]

    # add projection objective
    solver.Minimize(matmul(matminus(J, K), np.ones((n, n))))
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        res_K = [[el.solution_value() for el in row] for row in K] 
        new_J = matminus(res_K, np.diag(np.diag(res_K)))
        new_h = np.diag(res_K)
        print("Optimal value: %s" % solver.Objective().Value())
        return new_J, new_h
    else:
        print('The problem does not have an optimal solution.')
        return None

In [5]:
N = np.genfromtxt('newyork_travel_numbers.csv', delimiter=',')[:300, :300]
mu = 0.0023
h_max = -0.0001
J = N * np.log(1 / (1 - mu))
h = h_max * np.ones(len(N))
start_time = time.time()
result0 = ortools_map_lp(J, h, [0])
end_time = time.time()
print('Solved MAP LP in {} sec.'.format(end_time-start_time))
print(result0)

Solved MAP LP in 322.0834834575653 sec.
[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0],

In [6]:
# here we compute l1 projection onto the Safe Polytope
start_time = time.time()
projected_J, projected_h = l1_project(J, h, secure=1, margin=0.0001)
end_time = time.time()
print('Found projection in {} sec.'.format(end_time-start_time))

Optimal value: 3963.715690082605
Found projection in 802.4475154876709 sec.


In [7]:
start_time = time.time()
result0 = ortools_map_lp(np.array(projected_J), np.array(projected_h), [0])
end_time = time.time()
print('Solved MAP LP in {} sec.'.format(end_time-start_time))
print(result0)

Solved MAP LP in 0.4059596061706543 sec.
[[0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]